# Импортируем нужные библиотеки

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import model_selection, datasets, metrics, tree, ensemble
from scipy.interpolate import griddata

import lightgbm as lgb
import numpy as np
import pandas as pd
import seaborn as sns

import time

# Готовим данные

Генерируем выборку с помощью sklearn.datasets.make_regression (https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html) со следующими параметрами:
1. Наблюдений - 10 тысяч
2. Всех фичей - 100
3. Информативных фичей - 50

In [ ]:
regression_problem = datasets.make_regression(n_samples=10000, n_features=100,
                                              n_informative=50, random_state=42)

In [ ]:
regression_problem[0].shape

Разбиваем выборку на **train** и **test** в отношении 75:25.

In [ ]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(regression_problem[0],
                                                                                    regression_problem[1],
                                                                                    test_size=0.25,
                                                                                    random_state=42)

# Строим модели

Рассматривается задача **регрессии**: по исходным признакам требуется восстановить непрерывный таргет.

**Цель**: минимизировать метрику **MSE** (среднеквадратическая ошибка).

В качестве максимальной ошибки будем рассматривать алгоритм, который выдает в качестве ответа константу - среднее значение таргета на обучающей выборке.

In [ ]:
round(metrics.mean_squared_error(test_labels, test_labels.shape[0] * [train_labels.mean()]))

## Дерево решений

***Задание 1.***

Требуется обучая деревья решений на трейне, подобрать гиперпараметры **max_depth** и **min_samples_leaf** так, чтобы **MSE** на тесте было меньше **130 000**. При этом разница в качестве на обучающей и тестовой выборках должно различаться не более, чем на **15 000**.

In [ ]:
tree_regressor = tree.DecisionTreeRegressor(max_depth=..., min_samples_leaf=...,
                                            random_state=42)
tree_regressor.fit(train_data, train_labels)

In [ ]:
predictions = tree_regressor.predict(train_data)
train_tree_MSE = metrics.mean_squared_error(train_labels, predictions)

In [ ]:
predictions = tree_regressor.predict(test_data)
test_tree_MSE = metrics.mean_squared_error(test_labels, predictions)

In [ ]:
if (test_tree_MSE < 130000) and (test_tree_MSE - train_tree_MSE <= 15000):
    print('Вы восхитительны!')

***Задание 2.***

Расскажите, почему вы выбрали именно эти значения для гиперпараметров?

Ответ:

***Задание 3.***

Визуализируйте построенное дерево. Можно воспользоваться методом, который был продемонстрирован на семинаре.

***Задание 4.***

Далее посмотрим, как работает построенное дерево. Для этого выберем 2 самых значимых для дерева признака и визуализируем сплиты по ним. Можете использовать как указанную ниже функции функции, так и собственно написанные.

Нужно выбрать 2 признака и выполнить функцию для обучающей и тестовой выборок.

In [ ]:
def plot_decision_surface(i, j, data, labels, split_1, split_2, left_flg):
    """
        i - номер первого признака
        j - номер второго признака
        data - выборка с признаками
        labels - выборка с таргетом
        split_1 - значение из условия на первый признак
        split_2 - значение из условия на второй признак
        left_flg - флаг того, что второй признак рассматривается при невыполнении
        в дереве условия на первый признак
    """
    x = data[:, i]
    y = data[:, j]
    z = labels
    
    resolution = '50j'
    X, Y = np.mgrid[min(x):max(x):complex(resolution), min(y):max(y):complex(resolution)]
    points = [[a,b] for a,b in zip(x,y)]
    Z = griddata(points, z, (X, Y), method=contour_method)

    plt.figure(figsize=(14, 10))
    plt.contourf(X, Y, Z, cmap='winter')
    plt.colorbar()
    plt.plot([split_1, split_1], [y.min(), y.max()], 'r', linewidth=5, label='First split')
    if left_flg:
        split_list = [x.min(), split_1]
    else:
        split_list = [split_1, x.max()]
    plt.plot(split_list, [split_2, split_2], 'm', linewidth=5, label='Second split')
    plt.legend()
    plt.xlabel(f'X[{i}]')
    plt.ylabel(f'X[{j}]')
    plt.show()

In [ ]:
first_feature_number = ...
second_feature_number = ...

In [ ]:
plot_decision_surface(first_feature_number, second_feature_number, train_data, train_labels, ..., ..., ...)

In [ ]:
plot_decision_surface(first_feature_number, second_feature_number, test_data, test_labels, ..., ..., ...)

Как вы оцениваете разбиение выборок по двум сплитам из дерева?

Ответ:

## Случайный лес

***Задание 5.***

Задание, аналогичное второму:

Требуется обучая случайные леса на трейне, подобрать гиперпараметры **n_estimators**, **max_depth** и **min_samples_leaf** так, чтобы **MSE** на тесте было меньше **100 000**. При этом разница в качестве на обучающей и тестовой выборках должно различаться не более, чем на **15 000**.

In [ ]:
forest_regressor = ensemble.RandomForestRegressor(n_estimators=..., max_depth=..., min_samples_leaf=...,
                                                  max_features='sqrt', random_state=42)
forest_regressor.fit(train_data, train_labels)

In [ ]:
predictions = forest_regressor.predict(train_data)
train_forest_MSE = metrics.mean_squared_error(train_labels, predictions)

In [ ]:
predictions = forest_regressor.predict(test_data)
test_forest_MSE = metrics.mean_squared_error(test_labels, predictions)

In [ ]:
if (test_forest_MSE < 100000) and (test_forest_MSE - train_forest_MSE <= 15000):
    print('Вы прекрасны!!')

***Задание 6.***

Расскажите, почему вы выбрали именно эти значения для гиперпараметров?

Ответ:

***Задание 7.***

Выведите топ-10 по значимости признаков построенного дерева.

Два самых значимых признака дерева решений из пятого задания присутствуют в списке выше?

Ответ:

## Градиентный бустинг LGBM

***Задание 8.***

Снова требуется обучая теперь уже градиентный бустинги на трейне, подобрать гиперпараметры **n_estimators**, **max_depth** и **learning_rate** так, чтобы **MSE** на тесте было меньше **42 000**. При этом разница в качестве на обучающей и тестовой выборках должно различаться не более, чем на **10 000**.

In [ ]:
LGBM_regressor = lgb.RandomForestRegressor(n_estimators=..., max_depth=..., min_samples_leaf=...,
                                           random_state=42)
LGBM_regressor.fit(train_data, train_labels)

In [ ]:
predictions = LGBM_regressor.predict(train_data)
train_LGBM_MSE = metrics.mean_squared_error(train_labels, predictions)

In [ ]:
predictions = LGBM_regressor.predict(test_data)
test_LGBM_MSE = metrics.mean_squared_error(test_labels, predictions)

In [ ]:
if (test_LGBM_MSE < 42000) and (test_LGBM_MSE - train_LGBM_MSE <= 10000):
    print('Вы чудесны!!!')

***Задание 9.***

Расскажите, почему вы выбрали именно эти значения для гиперпараметров?

Ответ:

***Задание 10.***

Выведите топ-10 по значимости признаков построенного бустинга.

Два самых значимых признака дерева решений из пятого задания присутствуют в списке выше? Насколько данный топ совпадает с топом, полученным для случайного леса?

Ответ:

***Задание 11.***

Не меняя гиперпараметры **max_depth** и **learning_rate**, постройте зависимость **MSE** от числа деревьев, обучая для каждого значения **n_estimators** новый бустинг. **MSE** нужно подсчитать для обучающей и тестовой выборок.

Изобразите полученные зависимости на графике (ось Оx - **n_estimators**, ось Оy - **MSE**). Должно получится 2 линии, каждая из которых соответствует качеству на трейне и тесте.

Какой вывод можно сделать из графика? Что будет, если выбрать **n_estimators** меньшее или большее относительно того, которое вы получили в десятом задании?

Ответ:

В случае возникновения вопросов по заданиям обращайтесь в общий чат.